# CDMS Match Up Demo
Cloud-based Data Matchup Service (CDMS) allows users to collocate satellite to in situ and satellite to satellite data. 

In this notebook, the in situ data subsetting and match-up features will be demonstrated.

In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd
from urllib.parse import urljoin, urlencode
import numpy as np

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.lines import Line2D

import cdms_reader

### List available satellite and in situ datasets

In [ ]:
response = requests.get("https://doms.jpl.nasa.gov/list").json()
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(response)[['title', 'iso_start', 'iso_end']]
df.sort_values('title')

In [ ]:
in_situ_list = []
# manually add in situ metadata to list 
in_situ_list.append({"title": "ICOADS Release 3.0", "iso_start": "2017-01-01T00:00:00+0000","iso_end": "2018-12-31T23:59:59+0000"})
in_situ_list.append({"title": "SAMOS", "iso_start": "2017-01-01T00:00:00+0000","iso_end": "2018-12-31T23:59:59+0000"})
in_situ_list.append({"title": "1021_atlantic", "iso_start": "2019-01-30T01:00:00+0000","iso_end": "2019-10-16T16:00:00+0000"})
in_situ_list.append({"title": "antarctic_circumnavigation_2019", "iso_start": "2019-01-19T00:00:00+0000","iso_end": "2020-08-15T00:00:00+0000"})
in_situ_list.append({"title": "atlantic_to_med_2019_to_2020", "iso_start": "2019-10-18T10:12:00+0000","iso_end": "2020-07-17T13:45:00+0000"})
in_situ_list.append({"title": "shark-2018", "iso_start": "2018-03-15T20:00:00+0000","iso_end": "2018-06-17T00:00:00+0000"})

insitu_df = pd.DataFrame(in_situ_list)[['title', 'iso_start', 'iso_end']]
insitu_df

### Subset in situ dataset and plot

In [ ]:
query_params = {
    'provider': 'Saildrone',
    'project': 'shark-2018',
    'startTime': '2018-04-01T00:00:00Z',
    'endTime': '2018-04-07T23:59:59Z',
    'bbox': '-180,-90,180,90',
    'platform': '3B',
    'minDepth': -5,
    'maxDepth': 5,
    'variable': 'mass_concentration_of_chlorophyll_in_sea_water',
    'columns': 'mass_concentration_of_chlorophyll_in_sea_water',
    'startIndex': 0,
    'itemsPerPage': 15000
}

In [ ]:
in_situ_url = 'https://nasa-cdms.saildrone.com/1.0/query_data_doms_custom_pagination'
full_in_situ_url = f'{in_situ_url}?{urlencode(query_params)}'
print(full_in_situ_url)
response = requests.get(full_in_situ_url).json()
print(f'Total number of points returned {response["total"]}')
insitu_df = pd.DataFrame(response["results"])
insitu_df.head() # show first few records

In [ ]:
# Simple plot of data
insitu_df.plot(x='longitude', y='latitude', kind='scatter', c='mass_concentration_of_chlorophyll_in_sea_water', colormap='YlOrRd')


In [ ]:
# Plot with a map 

# initialize an axis
fig, ax = plt.subplots(figsize=(15,5))

# plot map on axis
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#print(countries)
countries[countries['continent'] == 'North America'].plot(color='lightgrey', ax=ax)

# parse dates for plot's title
first = insitu_df['time'].min()
last = insitu_df['time'].max()

# plot points
insitu_df.plot(x='longitude', y='latitude', kind='scatter', 
        c='mass_concentration_of_chlorophyll_in_sea_water', colormap='YlOrRd', 
        title=f'{first} to {last}', 
        ax=ax)

# add grid
ax.grid(alpha=0.5)
plt.show()


### Run satellite to in situ matchup query
Match JPL-L4-MRVA-CHLA-GLOB-v3.0 with Saildrone White Shark Café

In [ ]:
matchup_url = 'https://doms.jpl.nasa.gov/match_spark'

In [ ]:
def execute_matchup_request(query_params, in_situ_variable_name=None):
    full_matchup_url = f'{matchup_url}?{urlencode(query_params)}'
    print(full_matchup_url)
    
    response = requests.get(full_matchup_url)
    response = response.json()
    
    primary_points = []
    secondary_points = []
    for primary_point in response['data']:
        for variable in primary_point['matches'][0]['secondary']:
            if in_situ_variable_name:
                if variable['variable_name'] == in_situ_variable_name:
                    primary_points.append((float(primary_point['lon']), float(primary_point['lat']), float(primary_point['primary'][0]['variable_value'])))
                    secondary_points.append((float(primary_point['matches'][0]['lon']), float(primary_point['matches'][0]['lat']), float(variable['variable_value'])))
            else:
                # pick the first variable
                primary_points.append((float(primary_point['lon']), float(primary_point['lat']), float(primary_point['primary'][0]['variable_value'])))
                secondary_points.append((float(primary_point['matches'][0]['lon']), float(primary_point['matches'][0]['lat']), float(variable['variable_value'])))
                break
    print(f'Total number of primary matched points {len(primary_points)}')
    print(f'Total number of secondary matched points {len(secondary_points)}')
    return primary_points, secondary_points, response['executionId']

In [ ]:
query_params = {
    'primary': 'JPL-L4-MRVA-CHLA-GLOB-v3.0',
    'secondary': 'shark-2018',
    'startTime': '2018-04-01T00:00:00Z',
    'endTime': '2018-04-01T23:59:59Z',
    'tt': 86400,  # Time tolerance in seconds
    'rt': 50000,  # Spatial tolerance in meters
    'b': '-140,10,-110,40',
    'platforms': '3B',
    'parameter': 'mass_concentration_of_chlorophyll_in_sea_water',
    'depthMin': -5,
    'depthMax': 5,
    'matchOnce': 'true',
    'resultSizeLimit': 100
}

In [ ]:
primary_points, secondary_points, execution_id = execute_matchup_request(query_params, query_params['parameter'])

In [ ]:
def plot_points(primary_points, secondary_points):
    plt.figure(figsize=(20,5), dpi=500) 
    min_lon = min([point[0] for point in primary_points])
    max_lon = max([point[0] for point in primary_points])
    min_lat = min([point[1] for point in primary_points])
    max_lat = min([point[1] for point in primary_points])
    basemap = Basemap(
        projection='mill',
        lon_0=180,
        llcrnrlat=min_lat - 50,
        urcrnrlat=max_lat + 50,
        llcrnrlon=min_lon - 50,
        urcrnrlon=max_lon + 50
    )
    basemap.drawlsmask(
        land_color='lightgrey',
        ocean_color='white',
        lakes=True
    )

    # transform coordinates
    x1, y1 = basemap([point[0] for point in primary_points], [point[1] for point in primary_points])  
    x2, y2 = basemap([point[0] for point in secondary_points], [point[1] for point in secondary_points])
    
    # Draw scatter points
    plt.scatter(x2, y2, 10, marker='o', color='Blue', label='Primary point')
    plt.scatter(x1, y1, 10, marker='*', color='Green', label='Secondary point')

    # transform input bbox
    bx, by = basemap([-140, -110], [40, 10])  
    
    # Draw user provided bounds
    plt.gca().add_patch(patches.Rectangle(
        (bx[0], by[1]), abs(bx[0] - bx[1]), abs(by[0] - by[1]), 
        linewidth=1, 
        edgecolor='r', 
        facecolor='none'
    ))
    
    # Show legend
    handles, labels = plt.gca().get_legend_handles_labels()
    bbox_legend = Line2D(
        [0], [0], 
        color='red', 
        linewidth=1, 
        linestyle='-', 
        label='User search domain'
    )
    handles.append(bbox_legend)
    plt.legend(loc='upper left', handles=handles)
    
    plt.show()

In [ ]:
plot_points(primary_points, secondary_points)

In [ ]:
def generate_diff_plot(primary_points, secondary_points, primary_name, secondary_name, units):
    diffs = [primary_point[2] - secondary_point[2] for primary_point, secondary_point in zip(primary_points, secondary_points)]
    
    plt.figure(figsize=(20,5), dpi=500) 
    min_lon = min([point[0] for point in primary_points])
    max_lon = max([point[0] for point in primary_points])
    min_lat = min([point[1] for point in primary_points])
    max_lat = min([point[1] for point in primary_points])
    basemap = Basemap(
        projection='mill',
        lon_0=180,
        llcrnrlat=min_lat - 10,
        urcrnrlat=max_lat + 10,
        llcrnrlon=min_lon - 10,
        urcrnrlon=max_lon + 10
    )
    basemap.drawlsmask(
        land_color='lightgrey',
        ocean_color='white',
        lakes=True
    )

    # transform coordinates
    x1, y1 = basemap([point[0] for point in primary_points], [point[1] for point in primary_points])  
    x2, y2 = basemap([point[0] for point in secondary_points], [point[1] for point in secondary_points])
    
    # Customize colormap/colorbar
    cmap = plt.cm.coolwarm
    # Draw scatter points
    sc = plt.scatter(x2, y2, 30, marker='o', c=diffs, alpha=0.7, cmap=cmap)

    cb = plt.colorbar(sc)
    cb.ax.set_title(units,fontsize=8)        
    plt.title(f'Difference plot between {primary_name} and {secondary_name}')

In [ ]:
generate_diff_plot(primary_points, secondary_points, query_params['primary'], query_params['secondary'], 'milligram m-3')

### Run satellite to in situ matchup query

Match MUR25-JPL-L4-GLOB-v04.2 with ICOADS Release 3.0

In [ ]:
query_params = {
    'primary': 'MUR25-JPL-L4-GLOB-v04.2',
    'secondary': 'ICOADS_JPL',
    'startTime': '2018-04-01T00:00:00Z',
    'endTime': '2018-04-01T23:59:59Z',
    'tt': 86400,  # Time tolerance in seconds
    'rt': 50000,  # Spatial tolerance in meters
    'b': '-140,10,-110,40',
    'platforms': '42',
    'parameter': 'sea_water_temperature',
    'depthMin': -5,
    'depthMax': 5,
    'matchOnce': 'true',
    'resultSizeLimit': 0
}

primary_points, secondary_points, execution_id = execute_matchup_request(query_params, query_params['parameter'])

In [ ]:
def fetch_result(execution_id, output_format, output_file, primary_variable, secondary_variable):
    match_up_results_csv = f'{output_file}.csv'
    response = requests.get("https://doms.jpl.nasa.gov/domsresults", params={"id": execution_id, "output": output_format})
    with open(output_file, mode='wb') as f:
        f.write(response.content)

    matches = cdms_reader.assemble_matches(output_file)

    cdms_reader.matches_to_csv(matches, match_up_results_csv)
    
    columns_to_include = ["PrimaryData_lon", "PrimaryData_lat", "PrimaryData_datetime", "SecondaryData_lon", "SecondaryData_lat", "SecondaryData_datetime"]
    for variable in primary_variable:
        columns_to_include.append(f'PrimaryData_{variable}')
    for variable in secondary_variable:
        columns_to_include.append(f'SecondaryData_{variable}')
    return pd.read_csv(match_up_results_csv, usecols=columns_to_include)

In [ ]:
df = fetch_result(execution_id, "NETCDF", "mur_to_icoads_results.nc", ["sea_surface_foundation_temperature"], ["sea_water_temperature"])
df

In [ ]:
def generate_scatter_plot(primary_points, secondary_points, primary_name, secondary_name, variable_name, units):
    x = np.array([point[2] for point in secondary_points])
    y = np.array([point[2] for point in primary_points])
    m, b = np.polyfit(x, y, 1)
    fig, ax = plt.subplots()
    ax.set_title(f'{variable_name} scatter\n{primary_name} vs. {secondary_name}')
    ax.set_xlabel("%s %s" % (secondary_name, units))
    ax.set_ylabel("%s %s" % (primary_name, units))
    ax.scatter(x, y)
    ax.plot(x, m*x+b);
    ax.plot([0,1],[0,1], transform=ax.transAxes)

In [ ]:
generate_scatter_plot(primary_points, secondary_points, query_params['primary'], query_params['secondary'], query_params['parameter'], '°C')

### Run satellite to in situ matchup query

Match ASCATB-L2-Coastal with SAMOS


In [ ]:
query_params = {
    'primary': 'ASCATB-L2-Coastal',
    'secondary': 'SAMOS',
    'startTime': '2017-05-01T00:00:00Z',
    'endTime': '2017-05-04T23:59:59Z',
    'tt': 43200,  # Time tolerance in seconds
    'rt': 50000,  # Spatial tolerance in meters
    'b': '-100,20,-90,30',
    'platforms': '30',
    'parameter': 'wind_speed',
    'depthMin': -10,
    'depthMax': 10,
    'matchOnce': 'true',
    'resultSizeLimit': 0
}

primary_points, secondary_points, execution_id = execute_matchup_request(query_params, query_params["parameter"])

In [ ]:
df = fetch_result(execution_id, "NETCDF", "ascat_to_samos_results.nc", [query_params["parameter"]], [query_params["parameter"], "depth"])
df

In [ ]:
generate_diff_plot(primary_points, secondary_points, query_params['primary'], query_params['secondary'], 'm s-1')

In [ ]:
generate_scatter_plot(primary_points, secondary_points, query_params['primary'], query_params['secondary'], query_params['parameter'], 'm s-1')

### Run satellite to satellite matchup query
Match SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5 with OISSS_L4_multimission_7day_v1

In [ ]:
query_params = {
    'primary': 'SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5',
    'secondary': 'OISSS_L4_multimission_7day_v1',
    'startTime': '2018-08-01T00:00:00Z',
    'endTime': '2018-08-02T00:00:00Z',
    'tt': 604800,  # Time tolerance in seconds
    'rt': 50000,  # Spatial tolerance in meters
    'b': '-100,20,-90,30',
    'platforms': '65',
    'parameter': 'sea_surface_salinity',
    'depthMin': -20,
    'depthMax': 10,
    'matchOnce': 'true',
    'resultSizeLimit': 0
}

primary_points, secondary_points, execution_id = execute_matchup_request(query_params)

In [ ]:
df = fetch_result(execution_id, "NETCDF", "smap_sss_to_oisss_results.nc", [query_params['parameter']], [query_params['parameter']])
df

In [ ]:
generate_diff_plot(primary_points, secondary_points, query_params['primary'], query_params['secondary'], '1e-3')

In [ ]:
generate_scatter_plot(primary_points, secondary_points, query_params['primary'], query_params['secondary'], query_params['parameter'], '1e-3')

### Run satellite to satellite matchup query

Match ASCATB-L2-Coastal with MUR25-JPL-L4-GLOB-v04.2

In [ ]:
query_params = {
    'primary': 'ASCATB-L2-Coastal',
    'secondary': 'MUR25-JPL-L4-GLOB-v04.2',
    'startTime': '2018-08-01T00:00:00Z',
    'endTime': '2018-08-02T00:00:00Z',
    'tt': 43200,  # Time tolerance in seconds
    'rt': 50000,  # Spatial tolerance in meters
    'b': '-100,20,-90,30',
    'platforms': '65',
    'depthMin': -20,
    'depthMax': 10,
    'matchOnce': 'true',
    'resultSizeLimit': 0
}

primary_points, secondary_points, execution_id = execute_matchup_request(query_params)

In [ ]:
df = fetch_result(execution_id, "NETCDF", "ascat_l2_to_mur_l4_results.nc", ["wind_speed", "wind_to_direction"], ["sea_surface_foundation_temperature"])
df